In [ ]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, GenerationConfig

In [ ]:
from esm.sdk import client

model = client(model="esm3-small-2024-08", url="https://forge.evolutionaryscale.ai", token="PQZTYWyur8fur6koyJ3Jb")

In [35]:
structure_prediction_config = GenerationConfig(
    track="structure",  # We want ESM3 to generate tokens for the structure track
    num_steps=8,
    temperature=0.7,
)

structure_prediction_prompt = ESMProtein(sequence="AAAGACAAAAACMAAAFA")

structure_prediction = model.generate(
    structure_prediction_prompt, structure_prediction_config
)

In [41]:
# structure_prediction

In [37]:
structure_prediction_chain = structure_prediction.to_protein_chain()

In [ ]:
structure_prediction_chain.to_pdb_string()

'ATOM      1  N   ALA A   1       3.843 -12.138  16.392  1.00  0.80           N  \nATOM      2  CA  ALA A   1       4.625 -11.125  15.688  1.00  0.80           C  \nATOM      3  C   ALA A   1       3.764 -10.372  14.677  1.00  0.80           C  \nATOM      4  CB  ALA A   1       5.821 -11.767  14.989  1.00  0.80           C  \nATOM      5  O   ALA A   1       4.282  -9.801  13.714  1.00  0.80           O  \nATOM      6  N   ALA A   2       3.018 -10.114  15.015  1.00  0.86           N  \nATOM      7  CA  ALA A   2       2.125  -9.375  14.125  1.00  0.86           C  \nATOM      8  C   ALA A   2       2.673  -7.982  13.831  1.00  0.86           C  \nATOM      9  CB  ALA A   2       0.729  -9.276  14.734  1.00  0.86           C  \nATOM     10  O   ALA A   2       2.235  -7.322  12.886  1.00  0.86           O  \nATOM     11  N   ALA A   3       3.679  -7.521  14.437  1.00  0.87           N  \nATOM     12  CA  ALA A   3       4.188  -6.188  14.125  1.00  0.87           C  \nATOM     13  C 